**Import des librairies**

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


**Test de récupération d'infos sur une page de review**

In [ ]:
url = 'https://www.trustpilot.com/review/www.dugood.org?page=1&stars=5'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

reviews = soup.find_all('div', class_='styles_reviewCardInner__EwDq2')
reviews[0].get_text()

In [5]:
# On va chercher le nombre de page à parcourir
nb_pages = int(soup.find("a", attrs = {'name' :"pagination-button-last"}).get_text())

In [6]:
# On va chercher le nom de l'entreprise
entreprise = soup.find("h1").find("span").get_text().strip()

In [7]:
reviews[0].find(class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text.strip()

'Ichoku'

In [8]:
reviews[0].find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua").get_text()

'NG'

In [9]:
int(reviews[0].find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l").get_text().split()[0])

2

In [10]:
reviews[0].find('time').get('datetime')

'2023-10-27T19:30:24.000Z'

In [11]:
reviews[0].find(class_="typography_heading-s__f7029 typography_appearance-default__AAY17").get_text()

'I recently re-financed a truck note…'

In [12]:
reviews[0].find(class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn").get_text()

'I recently re-financed a truck note with DuGood Credit Union. The process was smooth and easy, the loan officer was friendly and accommodating and we received a better rate and saved a year in terms of payments! Thanks for helping us save money.'

**Récupération des infos sur une page entière**

In [13]:
lieux,pseudos,nbs_review,dates_review,titres_review,textes_review = [],[],[],[],[],[]

for review in reviews :
  lieu = review.find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua").text.strip()
  lieux.append(lieu)

  pseudo = review.find(class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text.strip()
  pseudos.append(pseudo)

  nb_review = int(review.find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l").text.strip().split()[0])
  nbs_review.append(nb_review)

  date_review = review.find('time').get('datetime')
  dates_review.append(date_review)

  titre_review = review.find(class_="typography_heading-s__f7029 typography_appearance-default__AAY17").text.strip()
  titres_review.append(titre_review)

  texte_review = review.find(class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn").text.strip()
  textes_review.append(texte_review)

**Fonction pour récupérer les infos d'une entreprise**

In [17]:
# FONCTION POUR RECUPERER LE NOMBRE DE PAGE EN TRIANT PAR ETOILE

def nb_pages (url, star):
  # Récupère le nombre de page d'une URL filtrée sur un nombre d'étoile
  url_star = url+"?stars="+str(star)
  response_star = requests.get(url_star)
  soup_star = BeautifulSoup(response_star.text, 'html.parser')
  if (soup_star.find("a", attrs = {'name' :"pagination-button-last"})) is None :
    if (soup_star.find("a", attrs = {'name' :"pagination-button-5"})) is None :
      if (soup_star.find("a", attrs = {'name' :"pagination-button-4"})) is None :
        if (soup_star.find("a", attrs = {'name' :"pagination-button-3"})) is None :
          if (soup_star.find("a", attrs = {'name' :"pagination-button-2"})) is None :
            if (soup_star.find("a", attrs = {'name' :"pagination-button-1"})) is None :
              nb_page = 0
            else :
              nb_page = 1
          else :
            nb_page = 2
        else :
          nb_page = 3
      else :
        nb_page = 4
    else :
      nb_page = 5
  else :
    nb_page = int(soup_star.find("a", attrs = {'name' :"pagination-button-last"}).get_text())
  return nb_page



# FONCTION POUR RECUPERER LES INFOS SUR UNE PAGE EN FONCTION DE L'URL, NOMBRE D'ETOILE ET PAGE

def recup_review_page (url, etoile, page):
  # Récupère les infos d'une URL en fonction du nombre d'étoile, et du numéro de page
  notes,lieux,pseudos,nbs_review,dates_review,titres_review,textes_review = [],[],[],[],[],[],[]
  if page == 1 :
    url_page = url+'?stars='+str(etoile)
  else :
    url_page = url+'?page='+str(page)+"&stars="+str(etoile)
  #print(url_page)
  response = requests.get(url_page)

  soup_response = BeautifulSoup(response.text, 'html.parser')

  reviews_response = soup_response.find_all('div', class_='styles_reviewCardInner__EwDq2')

  for review in reviews_response :
    notes.append(etoile)

    lieu = review.find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua").text.strip()
    lieux.append(lieu)

    pseudo = review.find(class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text.strip()
    pseudos.append(pseudo)

    nb_review = int(review.find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l").text.strip().split()[0])
    nbs_review.append(nb_review)

    date_review = review.find('time').get('datetime')
    dates_review.append(date_review)

    titre_review = review.find(class_="typography_heading-s__f7029 typography_appearance-default__AAY17").get_text()
    titres_review.append(titre_review)

    if review.find(class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn") is None :
      textes_review.append("")
    else :
      texte_review = review.find(class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn").text.strip()
      textes_review.append(texte_review)

  df_page = pd.DataFrame(list(zip(notes,lieux,pseudos,nbs_review,dates_review,titres_review,textes_review)), columns=["Note","Lieu", "Pseudo", "Nb_Review","Date","Titre","Review"])
  return df_page


# FONCTION POUR RECUPERER TOUTES LES INFOS D'UNE URL
# UTILISE LES 2 FONCTIONS PRECEDENTES

def recup_review (url):
  df_review = pd.DataFrame(columns=["Note","Lieu", "Pseudo", "Nb_Review","Date","Titre","Review"])
  for etoile in range(1,6):
    nombre_pages = nb_pages(url, etoile)
    #print(nombre_pages)
    if nombre_pages > 0 :
      for page in range(1,nombre_pages+1) :
        df_page = recup_review_page (url, etoile, page)
        df_review_concat = pd.concat([df_review, df_page])
        df_review = df_review_concat
  df_review.reset_index(drop=True, inplace=True)
  return df_review

**Test sur l'entreprise DuGood Credit Union**

In [4]:
url = 'https://www.trustpilot.com/review/www.dugood.org'

In [18]:
df_page_review = recup_review(url)
df_page_review

,Note,Lieu,Pseudo,Nb_Review,Date,Titre,Review
0,1,US,Gerald Spence,2,2023-06-30T23:01:04.000Z,I own 1 arce of land on Toledo Bend…,I own 1 arce of land on Toledo Bend lake with ...
1,1,US,Gerald Spence,2,2023-06-23T20:58:34.000Z,Weaponized against ordinary hard…,Weaponized against ordinary hard working middl...
2,1,US,LE Roi De La JUNGLE,1,2023-06-26T12:21:21.000Z,Credit card,Credit card please I'm Adleradlervoltaire1gmai...
3,1,US,Linda,1,2023-05-04T23:44:08.000Z,If I could leave 0 stars,"If I could leave 0 stars, I would. The loan d..."
4,1,US,Tina,3,2021-03-08T22:36:36.000Z,You've got to be kidding me!!!,You've got to be kidding me!!!! $10.00 to get ...
...,...,...,...,...,...,...,...
2216,5,US,George,3,2019-12-06T20:25:29.000Z,Very easy,I was pleased with the easy way it was to get ...
2217,5,US,Lene,6,2019-12-06T19:56:30.000Z,Everything went very smooth,
2218,5,US,Dolores,1,2019-12-01T23:19:33.000Z,Tara with DuGood helped me with…,Tara with DuGood helped me with everything. Al...
2219,5,US,Martha,1,2019-12-01T19:50:05.000Z,I was treated very well and everything…,I was treated very well and everything needed ...


**Test de récupération d'infos sur une page de listing d'entreprise**